In [13]:
import pandas as pd
import xarray as xr
import numpy as np

In [14]:
ssrd_ds = xr.open_dataset(f'../data/CAMS/cams_forecast/cams_forecast_ssrd_2023.grib',engine='cfgrib')
temp_ds = xr.open_dataset(f'../data/CAMS/cams_forecast/cams_forecast_t2m_2023.grib',engine='cfgrib')

site_df = pd.read_csv('../data/DeDe_48stations/DeDe_48sites_metadata.csv')

In [ ]:
st = '2023-01-01'
en = '2023-12-31'
time = pd.date_range(st, en, freq='1D')

for i, row in site_df.iterrows():
    forecast_1hr_list = []
    forecast_15min_list = []
    
    for date in time:
        df = ssrd_ds.sel(time=f'{date.date()}', latitude=row['lat'], longitude=row['lng'], method='nearest').to_dataframe()

        df['ssrd'] = (df['ssrd'] - df['ssrd'].shift(1).fillna(0)) / 3600
        df['ssrdc'] = (df['ssrdc'] - df['ssrdc'].shift(1).fillna(0)) / 3600

        df_15min = df.copy()
        df_15min.set_index('valid_time', inplace=True)
        df_15min = df_15min.resample('15min').asfreq()
        df_15min['ssrd'].interpolate(method='linear', inplace=True)
        df_15min['ssrdc'].interpolate(method='linear', inplace=True)
        df_15min.reset_index(inplace=True)

        forecast_1hr_list.append(df)
        forecast_15min_list.append(df_15min)

    temp_df = temp_ds.sel(latitude=row['lat'], longitude=row['lng'], method='nearest').to_dataframe()
    temp_df = temp_df.drop(index='0 days 12:00:00', level='step')
    temp_df.reset_index(inplace=True)
    temp_df.drop(columns=['step','time','number', 'surface', 'latitude', 'longitude'], inplace=True)
    temp_15min = temp_df.copy()
    temp_15min.set_index('valid_time', inplace=True)
    temp_15min = temp_15min.resample('15min').asfreq()
    temp_15min['t2m'].interpolate(method='linear', inplace=True)

    forecast_1hr = pd.concat(forecast_1hr_list, axis=0)
    forecast_15min = pd.concat(forecast_15min_list, axis=0)

    forecast_1hr.reset_index(drop=True, inplace=True)
    forecast_1hr = forecast_1hr.merge(temp_df, on='valid_time', how='right')
    forecast_1hr.drop(columns=['time','number', 'surface', 'latitude', 'longitude'], inplace=True)
    forecast_1hr.rename(columns={'valid_time': 'Datetime'}, inplace=True)
    forecast_1hr.set_index('Datetime', inplace=True)
    forecast_1hr = forecast_1hr.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    forecast_1hr.reset_index(inplace=True)
    forecast_1hr = forecast_1hr[forecast_1hr['Datetime'].dt.year == 2023]
    forecast_1hr.to_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_1hr_2023_site{row["Site_id"]:02d}.csv', index=False)

    forecast_15min.reset_index(drop=True, inplace=True)
    forecast_15min = forecast_15min.merge(temp_15min, on='valid_time', how='right')
    forecast_15min.drop(columns=['time','number', 'surface', 'latitude', 'longitude'], inplace=True)
    forecast_15min.rename(columns={'valid_time': 'Datetime'}, inplace=True)
    forecast_15min.set_index('Datetime', inplace=True)
    forecast_15min = forecast_15min.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    forecast_15min.reset_index(inplace=True)
    forecast_15min = forecast_15min[forecast_15min['Datetime'].dt.year == 2023]
    forecast_15min.to_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_15min_2023_site{row["Site_id"]:02d}.csv', index=False)
